In [1]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
from tqdm import tqdm
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
import sys
import os
import glob

from fairgbm import FairGBMClassifier


sys.path.append('../scripts')
import data
import models
import utils

%load_ext autoreload
%autoreload 2

In [2]:
def run_trial(
        trial,
        scorer,
        X_train,
        Y_train,
        A_train,
        X_val,
        Y_val,
        A_val,
        model_class,
        param_space,
        random_state = None
):  
    params = {}
    for name, values in param_space.items():
        if values["type"] == "int":
            values_cp = {n: v for n, v in values.items() if n != "type"}
            params[name] = trial.suggest_int(name, **values_cp)
        elif values["type"] == "categorical":
            values_cp = {n: v for n, v in values.items() if n != "type"}
            params[name] = trial.suggest_categorical(name, **values_cp)
        elif values["type"] == "float":  # corrected this line
            values_cp = {n: v for n, v in values.items() if n != "type"}
            params[name] = trial.suggest_float(name, **values_cp)

    model = model_class(**params)
    if isinstance(model, FairGBMClassifier):
        model.fit(X_train, Y_train, constraint_group=A_train)
    else:
        model.fit(X_train, Y_train, A_train)
    Y_val_pred = model.predict(X_val)
    return scorer(Y_val, Y_val_pred, A_val)

In [3]:
def get_group_feature(dataset, X_train, X_val, X_test):
    if dataset == "german2":
        A_train = X_train.Gender.astype(str)
        A_val = X_val.Gender.astype(str) 
        A_test = X_test.Gender.astype(str)
    elif dataset == "adult":
        A_train = X_train.sex.astype(str)
        A_val = X_val.sex.astype(str)
        A_test = X_test.sex.astype(str)

    sensitive_map = dict([
        (attr, i)
        for i, attr in enumerate(A_train.unique())
    ])
    A_train = A_train.map(sensitive_map)
    A_val = A_val.map(sensitive_map)
    A_test = A_test.map(sensitive_map)
    return A_train, A_val, A_test

In [4]:
def eval_model(y_ground, y_prob, y_pred, A):
    acc = accuracy_score(y_ground, y_pred)
    roc = roc_auc_score(y_ground, y_prob)
    eq_loss = utils.equalized_loss_score(y_ground, y_prob, A)
    eod = utils.equal_opportunity_score(y_ground, y_pred, A)
    spd = utils.statistical_parity_score(y_ground, y_pred, A)
    return {
        "acc" : acc,
        "roc" : roc,
        "eq_loss" : eq_loss,
        "eod" : eod,
        "spd" : spd
    }

In [5]:
def get_model(model_name, random_state=None):
    if model_name == "XtremeFair":
        def model(**params):
            return models.XtremeFair(random_state= random_state, **params)
    elif model_name == "XtremeFair_grad":
        def model(**params):
            return models.XtremeFair(dual_learning="gradient", random_state= random_state, **params)
    elif model_name == "XGBClassifier":
        def model(**params):
            assert params["fair_weight"] == 0
            return models.XtremeFair(random_state= random_state, **params)
    elif model_name == "FairGBMClassifier":
        def model(**params):
            return FairGBMClassifier(random_state= random_state, **params)
    return model
    
def get_param_spaces(model_name):
    if model_name == "XtremeFair":
        return models.PARAM_SPACES["XtremeFair"]
    elif model_name == "XtremeFair_grad":
        return models.PARAM_SPACES["XtremeFair"]
    elif model_name == "XGBClassifier":
        return models.PARAM_SPACES["XGBClassifier"]
    elif model_name == "FairGBMClassifier":
        return models.PARAM_SPACES["FairGBMClassifier"]

In [6]:
def group_experiment(args):
    # create output directory if not exists
    if not os.path.exists(args["output_dir"]):
        os.makedirs(args["output_dir"])

    # clear best_params.txt if exists
    if os.path.exists(os.path.join(args["output_dir"], f"best_params.txt")):
        os.remove(os.path.join(args["output_dir"], f"best_params.txt"))

    results = []

    cat_features = data.CAT_FEATURES[args["dataset"]]
    num_features = data.NUM_FEATURES[args["dataset"]]
    col_trans = ColumnTransformer(
        [
            ("numeric", StandardScaler(), num_features),
            (
                "categorical",
                OneHotEncoder(
                    drop="if_binary", sparse_output=False, handle_unknown="ignore"
                ),
                cat_features,
            ),
        ],
        verbose_feature_names_out=False,
    )
    col_trans.set_output(transform="pandas")
    scorer = utils.get_combined_metrics_scorer(
        alpha=args["alpha"], performance_metric="acc", fairness_metric="eod"
    )

    for i in tqdm(range(10)):
        # Load and prepare data
        X_train, Y_train, X_val, Y_val, X_test, Y_test = data.get_fold(
            args["dataset"], i, 0
        )

        # Define sensitive attribute from gender and age
        A_train, A_val, A_test = get_group_feature(
            args["dataset"], X_train, X_val, X_test
        )

        preprocess = Pipeline([("preprocess", col_trans)])
        preprocess.fit(X_train)
        X_train = preprocess.transform(X_train)
        X_val = preprocess.transform(X_val)
        X_test = preprocess.transform(X_test)


        study = optuna.create_study(direction="maximize")
        objective = lambda trial: run_trial(
            trial,
            scorer,
            X_train,
            Y_train,
            A_train,
            X_val,
            Y_val,
            A_val,
            get_model(args["model_name"]),
            get_param_spaces(args["model_name"]),
            0,
        )
        study.optimize(objective, n_trials=args["n_trials"])

        # save best params
        with open(os.path.join(args["output_dir"], f"best_params.txt"), "a+") as f:
            f.write(str(study.best_params))
            f.write("\n")

        model = get_model(args["model_name"])(**study.best_params)
        if isinstance(model, FairGBMClassifier):
            model.fit(X_train, Y_train, constraint_group=A_train)
        else:
            model.fit(X_train, Y_train, A_train)
        y_prob = model.predict_proba(X_train)[:, 1]
        thresh = utils.get_best_threshold(Y_train, y_prob)
        y_prob_test = model.predict_proba(X_test)[:, 1]
        y_pred_test = y_prob_test > thresh

        metrics = eval_model(Y_test, y_prob_test, y_pred_test, A_test)
        results.append(metrics)

    results = pd.DataFrame(results)
    results.to_csv(os.path.join(args["output_dir"], "results.csv"))
    results.mean().to_csv(os.path.join(args["output_dir"], "results_mean.csv"))



In [13]:
def summarize(dataset_name):
    experiments = glob.glob(f"../results/group_experiment/{dataset_name}/*")
    results = []
    for experiment in experiments:
        df = pd.read_csv(os.path.join(experiment, "results.csv"))
        df["experiment"] = experiment.split("/")[-1]
        df["eq_loss"] = 1 - df["eq_loss"].abs()
        df["spd"] = 1 - df["spd"].abs()
        df["eod"] = 1 - df["eod"].abs()
        results.append(df.iloc[:, 1:])
    results = pd.concat(results)
    # for each experiment, calculate the mean and std of each metric
    results_mean = results.groupby("experiment").mean()
    results_std = results.groupby("experiment").std()
    
    # combine dataframes into one with reorganized columns
    results = pd.concat([results_mean, results_std], axis=1)
    results.columns = pd.MultiIndex.from_product([["mean", "std"], results_mean.columns])
    results = results.swaplevel(axis=1)
    results = results[["roc", "acc", "eod","eq_loss", "spd"]]
    results = results.round(3)
    print(results)



In [8]:
args = {
    "dataset" : "german2",
    "alpha" : 1,
    "output_dir" : "../results/group_experiment/german2/XtremeFair_1",
    "model_name" : "XtremeFair",
    "n_trials" : 100
}
group_experiment(args)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [07:19<00:00, 43.90s/it]


In [9]:
args = {
    "dataset" : "german2",
    "alpha" : 1,
    "output_dir" : "../results/group_experiment/german2/XtremeFair_grad",
    "model_name" : "XtremeFair_grad",
    "n_trials" : 100
    
}
group_experiment(args)

100%|██████████| 10/10 [11:14<00:00, 67.43s/it]


In [10]:
args = {
    "dataset" : "german2",
    "alpha" : 1,
    "output_dir" : "../results/group_experiment/german2/XGBClassifier",
    "model_name" : "XGBClassifier",
    "n_trials" : 100
}
group_experiment(args)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [07:20<00:00, 44.01s/it]


In [11]:
args = {
    "dataset" : "german2",
    "alpha" : 1,
    "output_dir" : "../results/group_experiment/german2/FairGBMClassifier",
    "model_name" : "FairGBMClassifier",
    "n_trials" : 100
}
group_experiment(args)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:36<00:00,  9.61s/it]


In [ ]:
args = {
    "dataset" : "german2",
    "alpha" : 0.75,
    "output_dir" : "../results/group_experiment/german2/XtremeFair_1_alpha_075",
    "model_name" : "XtremeFair",
    "n_trials" : 100
}
group_experiment(args)

 30%|███       | 3/10 [00:45<01:42, 14.70s/it]/home/giovani/hiaac/dual_fair_boost/notebooks/../scripts/models.py:45: RuntimeWarning: overflow encountered in exp
  predt = 1 / (1 + np.exp(-predt))
/home/giovani/hiaac/dual_fair_boost/notebooks/../scripts/models.py:30: RuntimeWarning: overflow encountered in exp
  predt = 1 / (1 + np.exp(-predt))
/home/giovani/hiaac/dual_fair_boost/notebooks/../scripts/models.py:37: RuntimeWarning: overflow encountered in exp
  predt = 1 / (1 + np.exp(-predt))
/home/giovani/hiaac/dual_fair_boost/notebooks/../scripts/models.py:70: RuntimeWarning: overflow encountered in exp
  predt = 1 / (1 + np.exp(-predt))
/home/giovani/hiaac/dual_fair_boost/notebooks/../scripts/models.py:91: RuntimeWarning: overflow encountered in exp
  predt = 1 / (1 + np.exp(-predt))
/home/giovani/hiaac/dual_fair_boost/notebooks/../scripts/models.py:45: RuntimeWarning: overflow encountered in exp
  predt = 1 / (1 + np.exp(-predt))
/home/giovani/hiaac/dual_fair_boost/notebooks/../scrip

In [ ]:
args = {
    "dataset" : "german2",
    "alpha" : 0.75,
    "output_dir" : "../results/group_experiment/german2/XtremeFair_grad_alpha_075",
    "model_name" : "XtremeFair_grad",
    "n_trials" : 100
    
}
group_experiment(args)

 70%|███████   | 7/10 [02:14<00:58, 19.49s/it]/home/giovani/anaconda3/envs/dual_fair_boost/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/giovani/anaconda3/envs/dual_fair_boost/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 10/10 [03:02<00:00, 18.28s/it]


In [14]:
summarize("german2")

                     roc           acc           eod        eq_loss         \
                    mean    std   mean    std   mean    std    mean    std   
experiment                                                                   
FairGBMClassifier  0.769  0.049  0.705  0.050  0.846  0.123   0.889  0.107   
XGBClassifier      0.784  0.045  0.705  0.067  0.890  0.072   0.909  0.090   
XtremeFair_1       0.786  0.066  0.731  0.069  0.918  0.073   0.911  0.075   
XtremeFair_grad    0.759  0.098  0.709  0.069  0.906  0.069   0.835  0.170   

                     spd         
                    mean    std  
experiment                       
FairGBMClassifier  0.856  0.113  
XGBClassifier      0.860  0.078  
XtremeFair_1       0.882  0.074  
XtremeFair_grad    0.884  0.061  
